**Nama : Geraldus Jeremy Chandra**

**NIM : 2702368236**

**INFORMASI AWAL DAN TUJUAN**

dataset yang digunakan merupakan dataset dengan topik hotel, berisikan kolom-kolom yang akan digunakan untuk mempridiksi apakah booking statusnya akan dicancel atau tidak. Akan dilakukan EDA dan Preprocessing data lalu modelling Random Forest dan XGboost. Dua model dilakukan untuk membandingkan mana yang lebih baik dan hasil yang lebih baik akan digunakan untuk soal berikutnya OOP dan Streamlit.

**EDA**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import joblib
import os

import library yang dibutuhkan

In [ ]:
df = pd.read_csv("Dataset_B_hotel.csv")

read dataset

In [ ]:
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0.0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0.0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0.0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0.0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0.0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     35368 non-null  object 
 6   required_car_parking_space            35005 non-null  float64
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

melakukan checking untuk tipe data dan jumlah column dan countnya

In [ ]:
df.isnull().sum()

,0
Booking_ID,0
no_of_adults,0
no_of_children,0
no_of_weekend_nights,0
no_of_week_nights,0
type_of_meal_plan,907
required_car_parking_space,1270
room_type_reserved,0
lead_time,0
arrival_year,0


melakukan checking jumlah null

In [ ]:
for i in df.columns:
    print(f"Value counts untuk {i}:")
    print(df[i].value_counts())
    print()

Value counts untuk Booking_ID:
Booking_ID
INN36275    1
INN00001    1
INN00002    1
INN00003    1
INN00004    1
           ..
INN00023    1
INN00022    1
INN00021    1
INN00020    1
INN00019    1
Name: count, Length: 32607, dtype: int64

Value counts untuk no_of_adults:
no_of_adults
2    23503
1     6909
3     2063
0      119
4       13
Name: count, dtype: int64

Value counts untuk no_of_children:
no_of_children
0     30185
1      1459
2       943
3        17
9         2
10        1
Name: count, dtype: int64

Value counts untuk no_of_weekend_nights:
no_of_weekend_nights
0    15169
1     8953
2     8177
3      137
4      119
5       33
6       18
7        1
Name: count, dtype: int64

Value counts untuk no_of_week_nights:
no_of_week_nights
2     10253
1      8563
3      7053
4      2676
0      2136
5      1453
6       172
7       102
8        60
10       58
9        32
11       15
15        9
12        9
14        7
13        4
17        3
16        2
Name: count, dtype: int64

Value cou

membuat for looping untuk mengecek tipe-tipe jawaban yang ada untuk setiap data dan untuk mendapatkan anomali pada kolom

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df = df.drop('Booking_ID', axis = 1)

melakukan dropna berdasarkan ratio data yang hilang dengan keseluruhan jumlah kolom. Kolom booking ID didrop karena konteksnya yang merupakan hanya penanda unique number dan tidak diperlukan.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32607 entries, 0 to 32606
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          32607 non-null  int64  
 1   no_of_children                        32607 non-null  int64  
 2   no_of_weekend_nights                  32607 non-null  int64  
 3   no_of_week_nights                     32607 non-null  int64  
 4   type_of_meal_plan                     32607 non-null  object 
 5   required_car_parking_space            32607 non-null  float64
 6   room_type_reserved                    32607 non-null  object 
 7   lead_time                             32607 non-null  int64  
 8   arrival_year                          32607 non-null  int64  
 9   arrival_month                         32607 non-null  int64  
 10  arrival_date                          32607 non-null  int64  
 11  market_segment_

**Preprocessing : Encoding and Splitting**

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df['booking_status'].replace('Not_Canceled',1, inplace=True)
df['booking_status'].replace('Canceled',0, inplace=True)

df['arrival_year'].replace('2017',0, inplace=True)
df['arrival_year'].replace('2018',1, inplace=True)

saya melakukan encoding label secara manual untuk kolom booking status dan arrival year, arrival_year saya ubah karena disini nilai 2017 dan 2018 bukanlah sebuah nominal melainkan sebuah konteks tahun 2017 atau 2018.

In [ ]:
df['booking_status'].astype(int)

,booking_status
0,1
1,1
2,0
3,0
4,0
...,...
32602,1
32603,0
32604,1
32605,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32607 entries, 0 to 32606
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          32607 non-null  int64  
 1   no_of_children                        32607 non-null  int64  
 2   no_of_weekend_nights                  32607 non-null  int64  
 3   no_of_week_nights                     32607 non-null  int64  
 4   type_of_meal_plan                     32607 non-null  object 
 5   required_car_parking_space            32607 non-null  float64
 6   room_type_reserved                    32607 non-null  object 
 7   lead_time                             32607 non-null  int64  
 8   arrival_year                          32607 non-null  int64  
 9   arrival_month                         32607 non-null  int64  
 10  arrival_date                          32607 non-null  int64  
 11  market_segment_

In [ ]:
input_df=df.drop('booking_status',axis=1)
output_df=df['booking_status']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(input_df, output_df, test_size = 0.2, random_state = 42)

melakukan splitting data dengan ratio train 80% dan test 20%

In [ ]:
obj_cols = ['type_of_meal_plan', 'room_type_reserved', 'market_segment_type']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(x_train[obj_cols])

x_train_obj = pd.DataFrame(
    encoder.transform(x_train[obj_cols]),
    columns=encoder.get_feature_names_out(obj_cols),
    index=x_train.index
)

x_test_obj = pd.DataFrame(
    encoder.transform(x_test[obj_cols]),
    columns=encoder.get_feature_names_out(obj_cols),
    index=x_test.index
)

x_train = pd.concat([x_train.drop(columns=obj_cols), x_train_obj], axis=1)
x_test = pd.concat([x_test.drop(columns=obj_cols), x_test_obj], axis=1)

membuat list untuk 3 tipe kolom yang perlu di one hot encoder. Menggunakan one hot encoder karena tipe kolom bersifat jenis-jenis.

In [ ]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6522 entries, 29160 to 26757
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          6522 non-null   int64  
 1   no_of_children                        6522 non-null   int64  
 2   no_of_weekend_nights                  6522 non-null   int64  
 3   no_of_week_nights                     6522 non-null   int64  
 4   required_car_parking_space            6522 non-null   float64
 5   lead_time                             6522 non-null   int64  
 6   arrival_year                          6522 non-null   int64  
 7   arrival_month                         6522 non-null   int64  
 8   arrival_date                          6522 non-null   int64  
 9   repeated_guest                        6522 non-null   int64  
 10  no_of_previous_cancellations          6522 non-null   int64  
 11  no_of_previous_bo

**MODELLING**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

RF_class = RandomForestClassifier()
RF_class.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_predict, labels=[0,1], target_names=['Canceled','Not Canceled']))


Classification Report

              precision    recall  f1-score   support

    Canceled       0.96      0.37      0.53      2107
Not Canceled       0.77      0.99      0.86      4415

    accuracy                           0.79      6522
   macro avg       0.86      0.68      0.70      6522
weighted avg       0.83      0.79      0.76      6522



melakukan modelling Random forestr untuk perbandingan dengan XGBOOST, menggunakan classification report mengambil accuracynya sebagai acuan perbandingan. Untuk defaultnya saya tidak menggunakan parameter apa-apa untuk randomforest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'n_estimators': [200, 300],
    'max_depth': [8, 10, 12, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'max_samples': [None, 0.7, 0.9]
}

In [ ]:
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=parameters,
    n_iter=30,
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

In [ ]:
rf_random.fit(x_train,y_train)
print("Tuned Hyperparameters :", rf_random.best_params_)
print("Accuracy :",rf_random.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Tuned Hyperparameters : {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_samples': None, 'max_features': None, 'max_depth': 20, 'criterion': 'gini', 'bootstrap': True}
Accuracy : 0.8866014951121334


untuk mendapat hasil yang lebih baik, saya melakukan tuning dengan ramdomizedsearch dengan paramater diatas.

In [ ]:
RF_class_best = RandomForestClassifier(n_estimators= 300, min_samples_split= 2, min_samples_leaf= 4, max_samples =None, max_features=None, max_depth= 20, criterion='gini', bootstrap= True)

In [ ]:
RF_class_best.fit(x_train,y_train)

RandomForestClassifier(max_depth=20, max_features=None, min_samples_leaf=4,
                       n_estimators=300)

In [ ]:
y_predict_best=RF_class_best.predict(x_test)

In [ ]:
print('\nClassification Report\n')
print(classification_report(y_test, y_predict_best,labels=[0,1], target_names=['Canceled','Not Canceled']))


Classification Report

              precision    recall  f1-score   support

    Canceled       0.88      0.79      0.83      2107
Not Canceled       0.90      0.95      0.93      4415

    accuracy                           0.90      6522
   macro avg       0.89      0.87      0.88      6522
weighted avg       0.90      0.90      0.89      6522



Menyesuaikan fitting model dengan parameter yang didapatkan dengan hyperparamter tuning. Hasilnya accuracy menjadi lebih baik

In [ ]:
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

XGB_class = XGBClassifier(objective = 'binary:logistic', learning_rate = 0.01, max_depth = 4)
XGB_class.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = XGB_class.predict(x_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.59      0.70      2107
           1       0.83      0.96      0.89      4415

    accuracy                           0.84      6522
   macro avg       0.85      0.77      0.80      6522
weighted avg       0.84      0.84      0.83      6522



melakukan hal yang serupa dengan random forest, modelling XGboost dengan parameter default, lalu melihat accuract dengan classification report

In [ ]:
parameters_xgboost = {
    'objective': ['binary:logistic', 'multi:softmax', 'multi:softprob'],
    'max_depth': [2, 4, 6, 8],
    'learning_rate': [0.01, 0.1]

}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
XGB_class2 = XGBClassifier()
XGB_class2 = GridSearchCV(XGB_class2 ,
                          scoring='accuracy',
                          param_grid = parameters_xgboost,
                          verbose=3,
                          cv=5)

In [ ]:
XGB_class2.fit(x_train,y_train)
print("Tuned Hyperparameters :", XGB_class2.best_params_)
print("Accuracy :",XGB_class2.best_score_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END learning_rate=0.01, max_depth=2, objective=binary:logistic;, score=0.749 total time=   0.1s
[CV 2/5] END learning_rate=0.01, max_depth=2, objective=binary:logistic;, score=0.758 total time=   0.1s
[CV 3/5] END learning_rate=0.01, max_depth=2, objective=binary:logistic;, score=0.753 total time=   0.1s
[CV 4/5] END learning_rate=0.01, max_depth=2, objective=binary:logistic;, score=0.753 total time=   0.1s
[CV 5/5] END learning_rate=0.01, max_depth=2, objective=binary:logistic;, score=0.753 total time=   0.1s
[CV 1/5] END learning_rate=0.01, max_depth=2, objective=multi:softmax;, score=nan total time=   0.0s
[CV 2/5] END learning_rate=0.01, max_depth=2, objective=multi:softmax;, score=nan total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=2, objective=multi:softmax;, score=nan total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=2, objective=multi:softmax;, score=nan total time=   0.0s
[CV 5/5] 

In [ ]:
XGB_class_best = XGBClassifier(objective = 'binary:logistic', learning_rate = 0.1, max_depth = 8)

In [ ]:
XGB_class_best.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_predict_tuned = XGB_class_best.predict(x_test)

In [ ]:
print('\nClassification Report\n')
print(classification_report(y_test, y_predict_tuned,labels=[0,1], target_names=['Canceled','Not Canceled']))


Classification Report

              precision    recall  f1-score   support

    Canceled       0.86      0.80      0.83      2107
Not Canceled       0.91      0.94      0.92      4415

    accuracy                           0.89      6522
   macro avg       0.89      0.87      0.88      6522
weighted avg       0.89      0.89      0.89      6522



Kembali melakukan hyperparameter tuning namun kali ini menggunakan gridsearch, dengan parameter yang optimal, fitting disesuaikan untuk modelnya dan hasilnya accuracy pun lebih baik

**KESIMPULAN**

kedua model bagus, namun Random forest sedikit lebih baik dibandingkan dengan xgboost accuracynya, kita bisa asumsikan bahwa dataset yang kita punya secara default sudah cukup ok sehingga perbedaan model yang digunakan tidak begitu signifikan. Maka dari itu kita akan fokus pada random forest untuk melanjutkan proses OOP hingga streamlit.